In [1]:
from surface import grammar;
from surface import converter
from collections import defaultdict
import ast

In [2]:
def get_parse(fn, conll):
    with open(fn, "r") as f:
        next(f)
        conll_parse = {}
        parse = next(f).strip()
        text = [n.strip() for n in parse.strip("[]").split(",")]
        text_parse = []
        for i, w_id in enumerate(text):
            conll_parse[i] = conll[w_id.split("_")[1]]
            text_parse.append(conll[w_id.split("_")[1]][1])
        return text_parse, conll_parse

In [3]:
def set_parse(fn, graph):
    with open(fn, "w+") as f:
        f.write("# IRTG unannotated corpus file, v1.0\n")
        f.write("# interpretation ud: de.up.ling.irtg.algebra.graph.GraphAlgebra\n")
        f.write(graph + "\n")
        f.write("(dummy_0 / dummy_0)\n")

In [231]:
TRAIN_FILE = "/home/adaamko/data/UD-train/ru_syntagrus-ud-train.conllu"
TEST_FILE = "/mnt/store/adaamko/srst2019/T1-test/ru_syntagrus-ud-test.conllu"

In [232]:
grammar.train_subgraphs(TRAIN_FILE, TEST_FILE)
grammar.train_edges(TRAIN_FILE, TEST_FILE)

In [233]:
SUBGRAPH_GRAMMAR_FILE = "train_subgraphs"
EDGE_GRAMMAR_FILE = "train_edges"
TERMINAL_FILE_ORIG = TEST_FILE
TERMINAL_FILE_MIXED = TEST_FILE

In [236]:
rules, _ = converter.extract_rules(TERMINAL_FILE_MIXED)
graphs, _, id_graphs= converter.convert(TERMINAL_FILE_MIXED)
_, sentences, _ = converter.convert(TERMINAL_FILE_ORIG)
conll = grammar.get_conll_from_file(TERMINAL_FILE_MIXED)
id_to_parse = {}
stops = []

In [ ]:
for sen_id in range(1966, len(rules)):
    print(sen_id)
    try:
        grammar_fn = open('dep_grammar_spec.irtg', 'w') 
        grammar.generate_grammar(SUBGRAPH_GRAMMAR_FILE, rules[sen_id], grammar_fn)
        grammar.generate_terminal_ids(conll[sen_id], grammar_fn)
        grammar_fn.close()
        set_parse("ewt_ones", id_graphs[sen_id])
        !timeout 70 java -Xmx32G -cp alto-2.3.6-SNAPSHOT-all.jar de.up.ling.irtg.script.ParsingEvaluator ewt_ones -g dep_grammar_spec.irtg -I ud -O string=toString -o surface_eval_ewt
        text_parse, conll_parse = get_parse("surface_eval_ewt", conll[sen_id])
        id_to_parse[sen_id] = (text_parse, conll_parse)
    except StopIteration:
        print("stop iteratioin")
        stops.append(sen_id)
        continue

In [13]:
with open("result_parse", "w+") as f:
    for i in id_to_parse:
        f.write(sentences[i] + "\n")
        f.write(graphs[i] + "\n")
        f.write(str(id_to_parse[i]) + "\n")
        f.write("\n")

In [ ]:
for sen_id in stops:
    grammar_fn = open('dep_grammar_edges.irtg', 'w') 
    grammar.generate_grammar(EDGE_GRAMMAR_FILE, rules[sen_id], grammar_fn)
    grammar.generate_terminal_ids(conll[sen_id], grammar_fn)
    grammar_fn.close()
    set_parse("ewt_ones", id_graphs[sen_id])
    !java -Xmx32G -cp alto-2.3.6-SNAPSHOT-all.jar de.up.ling.irtg.script.ParsingEvaluator ewt_ones -g dep_grammar_edges.irtg -I ud -O string=toString -o surface_eval_ewt
    text_parse, conll_parse = get_parse("surface_eval_ewt", conll[sen_id])
    id_to_parse[sen_id] = (text_parse, conll_parse)

In [129]:
len(sentences)

0

In [ ]:
print(text_parse)
print(conll_parse)

In [ ]:
graphs[sen_id]

In [ ]:
id_to_parse[254]

In [ ]:
rules[sen_id]

In [ ]:
!java -Xmx32G -cp alto-2.3.6-SNAPSHOT-all.jar de.up.ling.irtg.script.ParsingEvaluator ewt_ones -g binarized.irtg -I ud -O string=toString -o surface_eval_ewt

In [125]:
text_parse, conll_parse = get_parse("surface_eval_ewt", conll[sen_id])

In [42]:
id_to_parse[sen_id] = (text_parse, conll_parse)

In [243]:
sen_id

1966

In [174]:
id_to_parse[sen_id] = ("", conll[sen_id])

In [240]:
len(graphs)

6491

In [230]:
with open("/mnt/store/adaamko/srst2019/test-results/hi_hdtb-ud-test.conllu" , "w") as f:
    for i in id_to_parse:
        conll_f = id_to_parse[i][1]
        for line in conll_f:
            f.write(str(line) + "\t")
            f.write("\t".join(conll_f[line]))
        f.write("\n")

In [241]:
sen_id

1811

In [ ]:
converter.to_tokenized_output("/mnt/permanent/home/judit/data/SRST/en_dev/", "/mnt/store/adaamko/srst2019/dev-results/")
converter.to_tokenized_output("/home/adaamko/data/UD-dev/", "/mnt/store/adaamko/srst2019/dev-golds/")

In [2]:
converter.to_tokenized_output("/home/adaamko/data/UD-dev/", "/mnt/store/adaamko/srst2019/dev-golds/")

In [3]:
converter.to_tokenized_output("/mnt/permanent/home/judit/data/SRST/en_dev/", "/mnt/store/adaamko/srst2019/dev-results/")


In [ ]:
stops

In [227]:
len(stops)

59

In [229]:
len(id_to_parse)

1684